Wildfire season started in Washington state in April this year.

See [Mandel et al. (2008)](https://doi.org/10.1016/j.matcom.2008.03.015) and [Reisch et al. (2024)](https://doi.org/10.1016/j.camwa.2024.01.024).

In [ ]:
from tqdm.notebook import trange, tqdm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import firedrake
from firedrake import Constant, inner, grad, dx, exp
import irksome
from irksome import Dt

In [ ]:
nx = 128
lx = 1000.0
mesh = firedrake.SquareMesh(nx, nx, lx, diagonal="crossed")
cg = firedrake.FiniteElement("CG", "triangle", 1)
b1 = firedrake.FiniteElement("Bernstein", "triangle", 2)
Q = firedrake.FunctionSpace(mesh, b1)

The reaction rate depends on two parameters, a critical temperature $T_c$ and a temperature scale $\Delta T$.
The rate is zero below the critical temperature and smoothly approaches 1 above.
The temperature scale dictates how fast the reaction rate approaches its maximum.
The explicit form of the rate is
$$R(T) = \exp\left(-\frac{\Delta T}{T - T_c}\right).$$
If the temperature exceeds the critical temperature by a factor of, say, $5\Delta T$, then the rate is more or less indistinguishable from its maximum value of 1.

We want to be careful about the numerics when we implement this equation.
Suppose the temperature is equal to $T_c + \delta T$ where $\delta T$ is very small.
In effect we're then trying to evaluate $\exp(-1 / \epsilon)$.
Depending on how the exponential function is implemented, either this or its derivative could become an unholy mess.

In [ ]:
T_c = Constant(100.0)
ΔT = Constant(100.0)

def reaction_rate(T):
    return firedrake.conditional(T <= T_c, 0, exp(-ΔT / (T - T_c)))

In [ ]:
x = firedrake.SpatialCoordinate(mesh)
T_a = Constant(15.0)
f = Constant(2.0)
δT = T_c + f * ΔT - T_a
Lx = Constant(lx)
expr = T_a + f * δT * x[0] / Lx
T = firedrake.Function(Q).interpolate(expr)

In [ ]:
r = firedrake.Function(Q).interpolate(reaction_rate(T))
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(r, axes=ax, cmap="inferno")
fig.colorbar(colors);

The other coefficients we need are the heat released per unit mass of fuel $Q$, the consumption rate $C$ of fuel, the ambient air temperature $T_a$, and the exchange coefficient $H$ with the upper atmosphere.

In [ ]:
ρ = Constant(40.0)      # material density:        kg / m³
c = Constant(1.0)       # specific heat capacity:  kJ / kg / °C
h = Constant(4.0)       # convection coefficient:  kW / m³ / °C
k = Constant(2e0)       # thermal conductivity:    kW / m / °C
E = Constant(4e3)       # heat release of burning: kJ / kg
τ = Constant(20.0)      # rate of burning:         s
T_a = Constant(30.0)    # ambient temperature:     °C
T_c = Constant(130.0)   # ignition temperature:    °C
ΔT = Constant(200.0)    # activation temperature:  °C

In [ ]:
x_0 = Constant((Lx / 2, Lx / 2))
r = Constant(Lx / 6)
T_f = Constant(200.0)
expr = T_a + f * (T_f - T_a) * exp(-inner(x - x_0, x - x_0) / r**2)

In [ ]:
Z = Q * Q
z = firedrake.Function(Z)
z.sub(0).interpolate(expr)
z.sub(1).assign(1.0)

lower = firedrake.Function(Z)
upper = firedrake.Function(Z)
lower.sub(0).assign(T_a)
upper.sub(0).assign(+np.inf)
lower.sub(1).assign(0.0)
upper.sub(1).assign(1.0);

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(z.sub(0), axes=ax, cmap="inferno")
fig.colorbar(colors);

In [ ]:
T, F = firedrake.split(z)
ϕ, ψ = firedrake.TestFunctions(Z)

R = firedrake.conditional(T <= T_c, 0, exp(-ΔT / (T - T_c)))

u = Constant((0.0, 0.0))
G_transport = ρ * c * (Dt(T) + inner(u, grad(T))) * ϕ * dx
G_diffusion = k * inner(grad(T), grad(ϕ)) * dx
G_reaction = (ρ * E * R * F / τ - h * (T - T_a)) * ϕ * dx
G_burning = (Dt(F) + R * F / τ) * ψ * dx

G = G_transport + G_diffusion - G_reaction + G_burning

In [ ]:
bc = firedrake.DirichletBC(Z.sub(0), T_a, "on_boundary")

method = irksome.BackwardEuler()
t = Constant(0.0)
dt = Constant(1.0)

params = {
    "solver_parameters": {
        "snes_monitor": ":wildfires.log",
        "snes_type": "vinewtonrsls",
        "snes_linesearch_type": "l2",
    },
    "stage_type": "value",
    "basis_type": "Bernstein",
    "bounds": ("stage", lower, upper),
    "bcs": [bc],
}
solver = irksome.TimeStepper(G, method, t, dt, z, **params)

In [ ]:
final_time = 1000.0
num_steps = int(final_time / float(dt))
Ts = [z.subfunctions[0].copy(deepcopy=True)]
Fs = [z.subfunctions[1].copy(deepcopy=True)]

output_freq = 2
for step in trange(num_steps):
    solver.advance()
    if (step + 1) % output_freq == 0:
        Ts.append(z.subfunctions[0].copy(deepcopy=True))
        Fs.append(z.subfunctions[1].copy(deepcopy=True))

In [ ]:
%%capture

fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True)
for ax in axes:
    ax.set_aspect("equal")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

axes[0].set_title("Temperature")
axes[1].set_title("Fuel fraction")
nsp = {"num_sample_points": 4}
t_kw = {"cmap": "inferno", "vmin": 30, "vmax": 900}
t_colors = firedrake.tripcolor(Ts[0], axes=axes[0], **t_kw, **nsp)
fig.colorbar(t_colors, orientation="horizontal")
f_kw = {"cmap": "Greens", "vmin": 0, "vmax": 1}
f_colors = firedrake.tripcolor(Fs[0], axes=axes[1], **f_kw, **nsp)
fig.colorbar(f_colors, orientation="horizontal")

fn_plotter = firedrake.FunctionPlotter(mesh, **nsp)
def animate(fields):
    T, F = fields
    t_colors.set_array(fn_plotter(T))
    f_colors.set_array(fn_plotter(F))

In [ ]:
animation = FuncAnimation(fig, animate, tqdm(list(zip(Ts, Fs))), interval=1e3/30)

In [ ]:
HTML(animation.to_html5_video())